# sdmath

Here we are comparing the simple math behind a typical noise-diode based ON-OFF position switch calibration.

The math below was taken from an earlier discussion in GBTOY.

$$
T_a(\nu) = T_{sys}^{ref} \times {  {sig(\nu) - ref(\nu)} \over { ref(\nu) }}          \tag{2}
$$
in Braatz(2009) the average is computed as follows:
$$
sig = {1\over 2} (sig_{calon} + sig_{caloff}), ref = {1\over 2} (ref_{calon} + ref_{caloff}) \tag{2a}
$$
whereas Teuben thought this would be the more logical one:
$$
T_{calon} =  T_{sys}^{ref} \times  {  {sig_{calon}(\nu) - ref_{calon}(\nu)} \over { ref_{calon}(\nu) }} \tag{2b}
$$
$$
T_{caloff} = T_{sys}^{ref} \times  {  {sig_{caloff}(\nu)- ref_{caloff}(\nu)} \over { ref_{caloff}(\nu) }}  \tag{2c}
$$
$$
T_a(\nu) = {1\over 2} (T_{calon} + T_{caloff})  \tag{2d}
$$


## Symbolic

```
If you write (S=sky  G=galaxy  N=noise diode) - sky should be S_on and S_off
   sig_calon   = S + G + N
   sig_caloff  = S + G
   ref_calon   = S     + N
   ref_caloff  = S

Braatz:    G / (S + N/2)

Teuben:    (G/S + G/(S_N) )/2

and their ratio

           T/B ~ 1 - (N/S)^2

which is about 8% (ngc5291)
```

In [ ]:
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from scipy.stats import norm
from astropy.io import fits
import astropy.units as u


In [ ]:
from dysh.fits.sdfitsload import SDFITSLoad
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data
from dysh.util.selection import Selection
from dysh.spectra.core import mean_tsys



In [ ]:
f1 = dysh_data(example="getps")  # NGC5291
print(f1)

In [ ]:
f2 = dysh_data(example="getps2") # NGC2415
print(f2)

In [ ]:
f3 = dysh_data('ngc5291.fits')
print(f3)

In [ ]:
sdf1 = GBTFITSLoad(f1)
sdf1.summary()

In [ ]:
sdf2 = GBTFITSLoad(f2)
sdf2.summary()

In [ ]:
sdf1.getspec(0).plot(xaxis_unit='chan')

In [ ]:
sdf2.getspec(0).plot(xaxis_unit='chan')

In [ ]:
# and older version of this file may fail due to missing ['PLNUM', 'SUBREF_STATE']
sdf3 = GBTFITSLoad(f3)
sdf3.summary()

the remainder is from `getps_dysh_vs_gbtidl.ipynb`


# Comparing `getps` between `Dysh` and `GBTIDL`.

In GBTIDL, from the `src/dysh/fits/tests/data` directory:
```
filein,"TGBT21A_501_11.raw.vegas.fits"
getps,152,ifnum=0,plnum=0,intnum=0
fileout,"TGBT21A_501_11_getps_scan_152_intnum_0_ifnum_0_plnum_0.fits"
keep
```
Look at the `TCAL` value and the computed `TSYS`:
```
print,!g.s[0].mean_tcal
   1.4551637
print,!g.s[0].tsys
   17.2400
```
`GBTIDL` seems to use a lower precision?
```
PRINT, SIZE(!g.s[0].tsys)
           0           4           1
```
Code 4 is for float, not double :)  [reference](https://www.l3harrisgeospatial.com/docs/size.html)

I tried figuring out why `GBTIDL` uses float instead of double, but could not. `GBTIDL` seems to load the data from the `SDFITS` without applying any type conversion, so I do not understand what happens.

So, let's check what happens if we re-scale the `dysh` results using a lower precison `TSYS` or the `TSYS` saved by `GBTIDL`.

Using the `TSYS` saved by GBTIDL makes the difference smaller. However, I still cannot figure out how to reproduce the `GBTIDL` value for `TSYS` starting from the raw data...

Also note that there is no .flag file here, so `GBTIDL` is not using any external flags.

**Solved:** The issues is that `IDL` uses inclusive ranges and `Python` does not. Increasing the upper edge to consider one extra channel makes the difference go away!

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from astropy.io import fits
from dysh.fits.gbtfitsload import GBTFITSLoad
import dysh.util as util

In [ ]:
testdata = util.get_project_testdata()
print(type(testdata))


In [ ]:
file1 = testdata / "TGBT21A_501_11/TGBT21A_501_11_getps_scan_152_intnum_0_ifnum_0_plnum_0.fits"
file2 = testdata / "TGBT21A_501_11/TGBT21A_501_11.raw.vegas.fits"

In [ ]:
hdu1 = fits.open(file1)
gbtidl = hdu1[1].data

In [ ]:
sdf = GBTFITSLoad(file2)
psscan = sdf.getps(152)
psscan.calibrate()
dysh_getps = psscan[0].calibrated(0).flux.to("K").value
dysh_getps_rescaled_low = dysh_getps/psscan[0]._tsys*17.2400   # Lower precision
dysh_getps_rescaled_sav = dysh_getps/psscan[0]._tsys*gbtidl["TSYS"] # Using the stored value

In [ ]:
print("GBTIDL vs Dysh native")
print("STD:", np.nanstd(gbtidl["DATA"][0] - dysh_getps))
print("MEDIAN:", np.nanmedian(gbtidl["DATA"][0] - dysh_getps))
print("\n")
print("GBTIDL vs Dysh rescaled to low precision")
print("STD:", np.nanstd(gbtidl["DATA"][0] - dysh_getps_rescaled_low))
print("MEDIAN:", np.nanmedian(gbtidl["DATA"][0] - dysh_getps_rescaled_low))
print("\n")
print("GBTIDL vs Dysh rescaled to saved TSYS")
print("STD:", np.nanstd(gbtidl["DATA"][0] - dysh_getps_rescaled_sav))
print("MEDIAN:", np.nanmedian(gbtidl["DATA"][0] - dysh_getps_rescaled_sav))

It is better to use the value stored by `GBTIDL` rather than what gets printed.

In [ ]:
psscan[0]._tsys

In [ ]:
gbtidl["TSYS"], gbtidl["TCAL"]

In [ ]:
print(psscan[0]._tsys/gbtidl["TSYS"] - 1.)

In [ ]:
# This is the difference we observe between `getps` in `GBTIDL` and `Dysh`.
print(psscan[0]._tsys/gbtidl["TSYS"] - 1.)

In [ ]:
# Let's try to reproduce the `TSYS` calculation.
hdu2 = fits.open(file2)
table2 = hdu2[1].data

In [ ]:
# Where are our sig-ref?
table2["CAL"], table2["PROCSEQN"], table2["OBSMODE"], table2["TCAL"]
# Why is `TCAL` different for the OFF position??

In [ ]:
data2 = table2["DATA"].astype(np.float64)

In [ ]:
# Unpack arrays.
nchan = data2.shape[1]
chi = int(0.1*nchan)
chf = int(0.9*nchan) + 1 + 1 # IDL uses inclusive channel ranges!
print("Channel range: ", chi, chf)
sig_on = data2[0,chi:chf]
sig_off = data2[1,chi:chf]
sig_tcal = table2["TCAL"][0]
ref_on = data2[2,chi:chf]
ref_off = data2[3,chi:chf]
ref_tcal = table2["TCAL"][2]

In [ ]:
tsys_sig = sig_tcal * np.nanmean(sig_off) / np.nanmean((sig_on - sig_off)) + sig_tcal/2.
tsys_ref = ref_tcal * np.nanmean(ref_off) / np.nanmean((ref_on - ref_off)) + ref_tcal/2.

print(tsys_sig, tsys_ref)
# Now they match!

In [ ]:
gbtidl["TSYS"], gbtidl["TCAL"], ref_tcal, sig_tcal

In [ ]:
print(f"{np.nanmean(ref_off):e}")

In [ ]:
np.nanmean(ref_off)

In [ ]:
np.nanmean(data2[3,3276:29492+1])

# Copy of our pytests
------------------

In [ ]:
def test_getps_single_int():

    gbtidl_file = util.get_project_testdata() / "TGBT21A_501_11/TGBT21A_501_11_getps_scan_152_intnum_0_ifnum_0_plnum_0.fits"
    hdu = fits.open(gbtidl_file)
    gbtidl_getps = hdu[1].data["DATA"][0]
    sdf_file = util.get_project_testdata() / "TGBT21A_501_11/TGBT21A_501_11.raw.vegas.fits"
    sdf = GBTFITSLoad(sdf_file)
    psscan = sdf.getps(152)
    assert len(psscan) == 1
    psscan.calibrate()
    dysh_getps = psscan[0].calibrated(0).flux.to("K").value
    diff = gbtidl_getps - dysh_getps
    assert np.nanmedian(diff) == 0.0
    assert np.all(abs(diff[~np.isnan(diff)]) < 5e-7)
    assert np.isnan(diff[3072])


In [ ]:
def test_gettp_single_int(debug=False):
    gbtidl_file = util.get_project_testdata() / "TGBT21A_501_11/TGBT21A_501_11_gettp_scan_152_intnum_0_ifnum_0_plnum_0_cal_state_1.fits"
    hdu = fits.open(gbtidl_file)
    gbtidl_gettp = hdu[1].data["DATA"][0]
    sdf_file = util.get_project_testdata() / "TGBT21A_501_11/TGBT21A_501_11.raw.vegas.fits"
    sdf = GBTFITSLoad(sdf_file)
    tps_on = sdf.gettp(152, sig=True, cal=True, calibrate=False)

    assert len(tps_on) == 1
    diff = tps_on[0].total_power(0).flux.value - gbtidl_gettp
    assert np.nanmean(diff) == 0.0
    tps_off = sdf.gettp(152, sig=True, cal=False, calibrate=False)
    assert len(tps_off) == 1
    gbtidl_file = util.get_project_testdata() / "TGBT21A_501_11/TGBT21A_501_11_gettp_scan_152_intnum_0_ifnum_0_plnum_0_cal_state_0.fits"
    hdu = fits.open(gbtidl_file)
    gbtidl_gettp = hdu[1].data["DATA"][0]
    diff = tps_off[0].total_power(0).flux.value - gbtidl_gettp
    assert np.nanmean(diff) == 0.0

    # Now, both on and off.
    tps = sdf.gettp(152, sig=True, cal=True)
    assert len(tps) == 1
    tps_tavg = tps.timeaverage()
    assert len(tps_tavg) == 1
    gbtidl_file = util.get_project_testdata() / "TGBT21A_501_11/TGBT21A_501_11_gettp_scan_152_ifnum_0_plnum_0.fits"
    hdu = fits.open(gbtidl_file)
    table = hdu[1].data
    spec = table["DATA"][0]
    diff = tps[0].total_power(0).flux.value - spec
    assert np.nanmean(diff) == 0.0
    if debug:
        return tps_on

    

In [ ]:
tps=test_gettp_single_int(True)
#tps.timeaverage()[0].plot()

In [ ]:
test_gettp_single_int(False)